## 1. Data Load

In [4]:
import pandas as pd 

# u.user 정보
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./data/u.user', sep='|', names=u_cols, encoding='latin-1')
users = users.set_index('user_id')
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [13]:
# u.item 정보
i_cols = ['movie_id', 'title', 'release data',' video release date', 'IMDB URL', 'unknown', 'Action', 'Adventure', 'Animation', 
          'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror' ,'Musical', 'Mystery',
          'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('./../data/u.item', sep='|', names=i_cols, encoding='latin-1')
movies = movies.set_index('movie_id')
movies.head(2)

,title,release data,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [60]:
# u.data 정보
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./../data/u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings = ratings.set_index('user_id')
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


## 2. Best-seller 추천

In [27]:
def recom_movie1(n_items):
    movie_sort = movie_mean.sort_values(ascending=False)[:n_items]
    recom_movies = movies.loc[movie_sort.index]
    recommendations = recom_movies['title']
    return recommendations

In [28]:
movie_mean = ratings.groupby(['movie_id'])['rating'].mean()
movie_mean

movie_id
1       3.878319
2       3.206107
3       3.033333
4       3.550239
5       3.302326
          ...   
1678    1.000000
1679    3.000000
1680    2.000000
1681    3.000000
1682    3.000000
Name: rating, Length: 1682, dtype: float64

In [29]:
recom_movie1(5)

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

In [30]:
def recom_movie2(n_items):
    return movies.loc[movie_mean.sort_values(ascending=False)[:n_items].index]['title']

In [32]:
recom_movie2(5)

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## 3. 정확도 측정

In [41]:
import numpy as np
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

In [42]:
len(ratings.index)

100000

In [43]:
len(set(ratings.index))

943

In [44]:
rmse = []
# 중복없는 사용자수
for user in set(ratings.index):
    y_true = ratings.loc[user]['rating']
    y_pred = movie_mean[ratings.loc[user]['movie_id']] # best-seller 방식에서는 영화의 평점평균이 예측값
    accuracy = RMSE(y_true, y_pred)
    rmse.append(accuracy)

In [52]:
len(rmse)

943

In [53]:
print(np.mean(rmse))

0.996007224010567


In [45]:
# 196번 사용자
ratings.loc[196].head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
196,393,4,881251863
196,381,4,881251728
196,251,3,881251274
196,655,5,881251793


## 4. 사용자 집단별 추천

In [71]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./../data/u.user', sep='|', names=u_cols, encoding='latin-1')

i_cols = ['movie_id', 'title', 'release data',' video release date', 'IMDB URL', 'unknown', 'Action', 'Adventure', 'Animation', 
          'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror' ,'Musical', 'Mystery',
          'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('./../data/u.item', sep='|', names=i_cols, encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./../data/u.data', sep='\t', names=r_cols, encoding='latin-1')

In [72]:
ratings = ratings.drop('timestamp', axis=1)
movies = movies[['movie_id', 'title']]

In [73]:
# train, test set 분리
from sklearn.model_selection import train_test_split
X = ratings.copy()
y = ratings['user_id']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [74]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

def score(model):
    id_pairs = zip(X_test['user_id'], X_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(X_test['rating'])
    return RMSE(y_true, y_pred)

In [76]:
X_train.head()

,user_id,movie_id,rating
61699,793,815,3
53857,624,150,4
48395,655,86,4
83829,893,759,3
68260,911,423,4


In [77]:
rating_matrix = X_train.pivot(index='user_id', columns='movie_id', values='rating')

In [78]:
rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1672,1673,1674,1675,1676,1677,1678,1679,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 4-1. 모델: trainset의 평점 평균으로 testset 평점 예측

In [83]:
def best_seller(user_id, movie_id):
    try:
        rating = train_mean[movie_id]
    except:
        # train set에 없지만, test set에 있는 영화는 3.0으로 처리함
        rating = 3.0
    return rating

In [84]:
train_mean = X_train.groupby(['movie_id'])['rating'].mean()

In [85]:
train_mean

movie_id
1       3.910714
2       3.258065
3       3.084507
4       3.571429
5       3.237288
          ...   
1677    3.000000
1678    1.000000
1679    3.000000
1681    3.000000
1682    3.000000
Name: rating, Length: 1638, dtype: float64

In [86]:
score(best_seller)

1.0219189418684351

### 4-2. 모델: trainset의 성별 평점 평균으로 testset 평점 예측

In [87]:
merged_ratings = pd.merge(X_train, users)

In [90]:
merged_ratings.head()

,user_id,movie_id,rating,age,sex,occupation,zip_code
0,793,815,3,22,M,student,85281
1,793,222,3,22,M,student,85281
2,793,273,3,22,M,student,85281
3,793,928,3,22,M,student,85281
4,793,109,4,22,M,student,85281


In [93]:
users = users.set_index('user_id')

In [98]:
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [95]:
g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()

In [96]:
g_mean

movie_id  sex
1         F      3.817073
          M      3.940945
2         F      3.466667
          M      3.217949
3         F      2.692308
                   ...   
1677      F      3.000000
1678      M      1.000000
1679      M      3.000000
1681      M      3.000000
1682      M      3.000000
Name: rating, Length: 3030, dtype: float64

In [101]:
def cf_gender(user_id, movie_id):
    if movie_id in rating_matrix:
        gender = users.loc[user_id]['sex']
        
        # 평가 인원이 적어서 특정 성별의 평점이 없을 수 있으므로
        if gender in g_mean[movie_id]:
            gender_rating = g_mean[movie_id][gender]
        else:
            gender_rating = 3.0
    else:
        gender_rating = 3.0
    
    return gender_rating

In [102]:
score(cf_gender)

1.0305284604296272